In [1]:
import requests as r
import sqlite3
import lxml
from bs4 import BeautifulSoup
import math
import time
import pprint
import threading

In [4]:
# index
index = "https://www.1111.com.tw/job-bank/job-index.asp?si=1&d0=140200,140300,140400,140100&fs=1&ps=100&page=2"
#        https://www.1111.com.tw/job-bank/job-index.asp?si=1&d0=140200,140300,140400,140100&fs=1&page=2

In [3]:
# create table & drop table
with sqlite3.connect('job.sqlite') as conn:
    c = conn.cursor()
    try:
        c.execute('drop table job1111')
        print('drop table job1111')
    except:
        c.execute('''
            CREATE TABLE job1111(
              jobID INTEGER PRIMARY KEY AUTOINCREMENT,
              title TEXT NOT NULL,
              href TEXT NOT NULL,
              times INTEGER,
              info TEXT)''')
        print('create table job1111')

create table job1111


In [5]:
# number of total job 
def getPage1111(href):
#     import locale
#     locale.setlocale(locale.LC_ALL, 'en_US.UTF-8') 
    res = r.get(href)
    
    return int(BeautifulSoup(res.text, 'lxml').select('div.pagedata')[0].text.split('頁')[0].split('/')[1].strip())

totalPages = getPage1111(index)
totalPages

80

In [6]:
res = r.get(index)
soup = BeautifulSoup(res.text, 'lxml').select('div#jobResult > ul')

In [116]:
# test program -> get href
soup[0].select('div.jbInfoin')[0].select('a')[0]['href'][2:]

'www.1111.com.tw/job/77820429/'

In [7]:
# sqlite insert&update -> job1111
def insert_href(title, href):
    try:
        with sqlite3.connect('job.sqlite') as conn:

            # Query histry, whether we have insert it before...
            c = conn.cursor()
            qryString = "SELECT href FROM job1111 where href=:href"
            c.execute(qryString, {'href' : href})
            # if there are nothing like href
            if len(c.fetchall()) == 0:
                # insert new one
                insert_string = "INSERT INTO job1111 (title, href, times) VALUES (?, ?, 1)"
                c.execute(insert_string, (title, href))
            else:
                update_string = "UPDATE job1111 SET times = times + 1 WHERE href = ?"
                c.execute(update_string, (href,))

    except ConnectionError as e:
        print(e)
        print(href)

In [8]:
# main program
for page in range(1, totalPages + 1):
# for page in range(1, 2):
    href = "https://www.1111.com.tw/job-bank/job-index.asp?si=1&d0=140200,140300,140400,140100&fs=1&ps=100&page={}".format(page)
    soup = BeautifulSoup(r.get(href).text, 'lxml')
    jidSoup = soup.select('div.jbInfo > div')
    totalJid = len(jidSoup)
    print('page: ' + str(page))
    for jid in range(0, totalJid):
        title = jidSoup[jid].select('h3')[0].text
        href = jidSoup[jid].select('a')[0]['href'][2:]
        insert_href(title, href)
    
#     time.sleep(2)
res.close()
print('done!!')

page: 1
page: 2
page: 3
page: 4
page: 5
page: 6
page: 7
page: 8
page: 9
page: 10
page: 11
page: 12
page: 13
page: 14
page: 15
page: 16
page: 17
page: 18
page: 19
page: 20
page: 21
page: 22
page: 23
page: 24
page: 25
page: 26
page: 27
page: 28
page: 29
page: 30
page: 31
page: 32
page: 33
page: 34
page: 35
page: 36
page: 37
page: 38
page: 39
page: 40
page: 41
page: 42
page: 43
page: 44
page: 45
page: 46
page: 47
page: 48
page: 49
page: 50
page: 51
page: 52
page: 53
page: 54
page: 55
page: 56
page: 57
page: 58
page: 59
page: 60
page: 61
page: 62
page: 63
page: 64
page: 65
page: 66
page: 67
page: 68
page: 69
page: 70
page: 71
page: 72
page: 73
page: 74
page: 75
page: 76
page: 77
page: 78
page: 79
page: 80
done!!


In [10]:
res.close()

In [9]:
# Query result program
# 2017/05/08 : ? web(exclude repeat)
with sqlite3.connect('job.sqlite') as conn:
    c = conn.cursor()
    pprint.pprint(list(c.execute('select * from job1111')))

[(1, '軟體工程師', 'www.1111.com.tw/job/77820429/', 2, None),
 (2, 'PHP伺服器開發工程師', 'www.1111.com.tw/job/79879687/', 2, None),
 (3, 'CIM Engineer', 'www.1111.com.tw/job/79863160/', 2, None),
 (4, '工程人員', 'www.1111.com.tw/job/79833278/', 2, None),
 (5, 'Java開發工程師(台北)', 'www.1111.com.tw/job/78593971/', 2, None),
 (6, 'FB企劃專員', 'www.1111.com.tw/job/79907356/', 1, None),
 (7, '【總公司】iOS APP程式設計師1', 'www.1111.com.tw/job/79907339/', 1, None),
 (8, '網頁設計人員', 'www.1111.com.tw/job/79907244/', 1, None),
 (9, '霹靂-資訊部-IT維護人員(虎尾)', 'www.1111.com.tw/job/79907149/', 1, None),
 (10, '電腦維護工程師', 'www.1111.com.tw/job/79906972/', 1, None),
 (11, '網管工程師', 'www.1111.com.tw/job/79906971/', 1, None),
 (12, '雲端科技-系統工程師', 'www.1111.com.tw/job/79906959/', 1, None),
 (13, '知名電信-國際NOC機房維運助理', 'www.1111.com.tw/job/79906957/', 1, None),
 (14, '資訊類 -ERP課長/高級工程師', 'www.1111.com.tw/job/79906826/', 1, None),
 (15, '程式設計專責', 'www.1111.com.tw/job/79906792/', 1, None),
 (16, 'MIS主管', 'www.1111.com.tw/job/79906615/', 1, None),
 (17

  None),
 (1221,
  '106年研發替代役-加速器控制系統整合與診斷系統發展及維護',
  'www.1111.com.tw/job/79739992/',
  1,
  None),
 (1222,
  '106年研發替代役-TPS 25A同調Ｘ光散射-晶體缺陷量測技術研發',
  'www.1111.com.tw/job/79739989/',
  1,
  None),
 (1223, '106年研發替代役-TPS奈米級影像實驗站建造', 'www.1111.com.tw/job/79739983/', 1, None),
 (1224, '助理工程師', 'www.1111.com.tw/job/79739703/', 1, None),
 (1225, '106年研發替代役–多媒體軟體工程師', 'www.1111.com.tw/job/79739593/', 1, None),
 (1226,
  '106年研發替代役–Video / Audio Codec軟體工程師',
  'www.1111.com.tw/job/79739591/',
  1,
  None),
 (1227, '資訊室資訊管理師', 'www.1111.com.tw/job/79739324/', 1, None),
 (1228, '系統維護工程師', 'www.1111.com.tw/job/79739176/', 1, None),
 (1229, '系統維護人員（106年產業訓儲替代役專用）', 'www.1111.com.tw/job/79739175/', 1, None),
 (1230, 'NB相容性測試工程師_6075', 'www.1111.com.tw/job/79571614/', 1, None),
 (1231, '研發測試員', 'www.1111.com.tw/job/79735246/', 1, None),
 (1232, '資訊部副理(駐大陸肇慶廠)', 'www.1111.com.tw/job/79735214/', 1, None),
 (1233, '高級技術經理-73C60', 'www.1111.com.tw/job/79734933/', 1, None),
 (1234, '.NET Engineer(儲備專案工

  'Embedded Systems GUI Programmer (Qt)',
  'www.1111.com.tw/job/79897334/',
  1,
  None),
 (3362,
  'Embedded and IoT Application Engineer',
  'www.1111.com.tw/job/79897328/',
  1,
  None),
 (3363, 'DSP/FPGA Engineer', 'www.1111.com.tw/job/79897318/', 1, None),
 (3364, '軟體開發工程師', 'www.1111.com.tw/job/79897120/', 1, None),
 (3365, '聲學工程師_6425', 'www.1111.com.tw/job/79856912/', 1, None),
 (3366, '數位工程師', 'www.1111.com.tw/job/79896601/', 1, None),
 (3367, '測試人員(適用桃園安心就業讚方案)', 'www.1111.com.tw/job/79895613/', 1, None),
 (3368, '(新北市新莊區)資深應用系統工程師', 'www.1111.com.tw/job/79893506/', 1, None),
 (3369, '智能語意系統開發工程師(.NET)', 'www.1111.com.tw/job/79893457/', 1, None),
 (3370,
  '手機\\平板產品諮詢客服專員(PM1500~AM2400)',
  'www.1111.com.tw/job/79893446/',
  1,
  None),
 (3371,
  '派駐美國-Professional services engineer',
  'www.1111.com.tw/job/79892926/',
  1,
  None),
 (3372, '工程助理', 'www.1111.com.tw/job/79887923/', 1, None),
 (3373,
  'YS Sustaining Engineer-Mechatronics(MTD)',
  'www.1111.com.tw/job/79887318

  'Software Development Engineer (RD)',
  'www.1111.com.tw/job/79874124/',
  1,
  None),
 (5050, '網路行銷', 'www.1111.com.tw/job/79874004/', 1, None),
 (5051, '數據分析師', 'www.1111.com.tw/job/79873981/', 1, None),
 (5052, '光傳輸工程師', 'www.1111.com.tw/job/79873825/', 1, None),
 (5053, 'A160_軟體工程師_V0R300', 'www.1111.com.tw/job/79873797/', 1, None),
 (5054, 'A158_MIS軟體工程師_VF0300', 'www.1111.com.tw/job/79873796/', 1, None),
 (5055, 'A144_韌體研發工程師(MCU)_V0R300', 'www.1111.com.tw/job/79873775/', 1, None),
 (5056, 'Senior testing consultant', 'www.1111.com.tw/job/79873736/', 1, None),
 (5057, 'A142_韌體研發工程師(SSD)_V0R300', 'www.1111.com.tw/job/79873721/', 1, None),
 (5058, 'A121_硬體研發工程師(SSD)_V0R300', 'www.1111.com.tw/job/79873667/', 1, None),
 (5059, 'SAP系統管理師', 'www.1111.com.tw/job/79872380/', 1, None),
 (5060, '雲端服務後端工程師 - J1299', 'www.1111.com.tw/job/79872222/', 1, None),
 (5061, '行動應用軟體工程師 - J1300', 'www.1111.com.tw/job/79872219/', 1, None),
 (5062, '網路管理系統工程師', 'www.1111.com.tw/job/79871825/', 1, Non

  None),
 (6422, '電控設計編碼工程師', 'www.1111.com.tw/job/79576372/', 1, None),
 (6423, '(總部)R1900-應用軟體研發工程師', 'www.1111.com.tw/job/79432604/', 1, None),
 (6424,
  '工程類-User Experience Designer [台北]',
  'www.1111.com.tw/job/79428959/',
  1,
  None),
 (6425, '工程類-Digital Designer [台北]', 'www.1111.com.tw/job/79428954/', 1, None),
 (6426,
  'MKT類-Digital Marketing Manager [台北]',
  'www.1111.com.tw/job/79428948/',
  1,
  None),
 (6427, '105KU027-產業創新營運輔導工程師', 'www.1111.com.tw/job/79008199/', 1, None),
 (6428, 'PLC工程師', 'www.1111.com.tw/job/79004178/', 1, None),
 (6429, '(汐止廠)F1230-主機板維修工程師', 'www.1111.com.tw/job/78595791/', 1, None),
 (6430, '1. 新產品專案 經理', 'www.1111.com.tw/job/78499489/', 1, None),
 (6431, 'Asp.Net,C#程式設計師(資訊部)', 'www.1111.com.tw/job/78339844/', 1, None),
 (6432, '(總部)R1200-硬體研發主管', 'www.1111.com.tw/job/77789231/', 1, None),
 (6433, '研發類-CAD Engineer(11)', 'www.1111.com.tw/job/77335054/', 1, None),
 (6434, '(總部)R1900-BIOS (資深/主任)工程師', 'www.1111.com.tw/job/77302722/', 1, None),
 (

 (7655,
  'SU_Android Developer_知名新創公司擁有全球百萬次下載',
  'www.1111.com.tw/job/78352764/',
  1,
  None),
 (7656, 'IL- 知名外商誠徵Android工程師', 'www.1111.com.tw/job/78352197/', 1, None),
 (7657, 'KL_IOT智慧家居_SW manager_上市公司', 'www.1111.com.tw/job/78338613/', 1, None),
 (7658, 'AL-海外工作機會-技術支援工程師(FAE/菲律賓)', 'www.1111.com.tw/job/78260026/', 1, None),
 (7659, 'AL-海外工作機會-網路系統工程師(Linux系統)', 'www.1111.com.tw/job/78260023/', 1, None),
 (7660, 'AL-海外工作機會-JAVA Engineer', 'www.1111.com.tw/job/77491000/', 1, None),
 (7661, 'AL-知名網購平台-ASP.NET資深軟體開發人員', 'www.1111.com.tw/job/77490995/', 1, None),
 (7662, '全球ODM/OEM製造大廠_Server HW_LH', 'www.1111.com.tw/job/77828742/', 1, None),
 (7663,
  'IL-外商 iOS APP Developer 薪水優',
  'www.1111.com.tw/job/77815547/',
  1,
  None),
 (7664,
  'SU_Sr.PHP Engineer (新創團隊leader)',
  'www.1111.com.tw/job/77810662/',
  1,
  None),
 (7665,
  'SU_Sr.PHP Engineer知名社群網站200萬會員',
  'www.1111.com.tw/job/77810659/',
  1,
  None),
 (7666, 'AC*知名系統廠* 急徵 RD Leader 內湖區', 'www.1111.com.tw/job/77344179

In [45]:
# Query maximun times
with sqlite3.connect('job.sqlite') as conn:
    c = conn.cursor()
#     pprint.pprint(list(c.execute('select * from job1111 where times > 1')))
    pprint.pprint(list(c.execute('select count(times), sum(times) from job1111 where times > 1')))

[(0, None)]
